In [1]:
import cv2
import mediapipe as mp
import os

In [6]:
def initialize_mediapipe():
    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    return mp_hands, mp_drawing, hands

def procesar_frame(frame, hands):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = hands.process(image)
    return results

def calcular_diferencia(puntos_previos,puntos_actuales, handedness_prev, handedness_actual):
    if puntos_previos is None or puntos_actuales is None:
        return 10.0
    
    # Comprobamos si alguna de las listas está vacía o si las dos listas tienen diferentes longitudes
    if len(puntos_previos) != len(puntos_actuales):
        return 10.0

    num_hands_previas = len(puntos_previos)
    num_hands_actuales = len(puntos_actuales)

    if num_hands_previas != num_hands_actuales:
        return 10.0

    total_diff = 0.0
    if len(puntos_previos) == len(puntos_actuales) and len(puntos_actuales) == 2:
        if handedness_prev[0].classification[0].label != handedness_actual[0].classification[0].label:
            puntos_actuales = puntos_actuales[::-1]
    for i in range(num_hands_previas):
        hand_prev = puntos_previos[i].landmark
        hand_actual = puntos_actuales[i].landmark

        diff = sum([
            abs(lm1.x - lm2.x) + abs(lm1.y - lm2.y) + abs(lm1.z - lm2.z)
            for lm1, lm2 in zip(hand_prev, hand_actual)
        ])
        total_diff += diff
    print("Total dif", total_diff)
    if num_hands_previas >= 2:
        print(total_diff)
        total_diff /= num_hands_previas
    print(total_diff)
    return float(total_diff)

def extract_key_frames(video, threshold=4.4, min_frame_interval=3):
    mp_hands, mp_drawing, hands = initialize_mediapipe()
    puntos_previos = []
    handedness_prev = []
    key_frames = []
    frame_count = 0
    print("Iniciando la extracción de frames clave")

    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            print("Fin del video o error al leer frame")
            break
        
        frame_count += 1
        if len(key_frames)==0 or frame_count - key_frames[-1][1] > min_frame_interval:
            results = procesar_frame(frame, hands)
            
            if results.multi_hand_landmarks:
                puntos_actuales = results.multi_hand_landmarks
                handedness_actual = results.multi_handedness if results.multi_handedness else []
                diff = calcular_diferencia(puntos_previos, puntos_actuales, handedness_prev, handedness_actual)

                if diff > threshold:
                    key_frames.append((frame, frame_count))
                    puntos_previos = puntos_actuales
                    handedness_prev = handedness_actual
            else:
                puntos_previos = None

    video.release()
    return [frame for frame, _ in key_frames]

def save_key_frames(key_frames, output_folder):
    for idx, key_frame in enumerate(key_frames):
        cv2.imwrite(f'{output_folder}/key_frame_{idx}.png', key_frame)

In [7]:
video_path = 'C:\\Users\\48113164\\Documents\\GitHub\\SignAI-IA.dev\\recursos\\videoprueba1.mp4'
output_folder='C:\\Users\\48113164\\Documents\\GitHub\\SignAI-IA.dev\\recursos\\frames'
video = cv2.VideoCapture(video_path)
if not os.path.isfile(video_path):
    print(f"Error: El archivo {video_path} no existe.")
elif not video.isOpened():
    print("Error: No se pudo abrir el video.")
else:
    key_frames = extract_key_frames(video)
    save_key_frames(key_frames, output_folder)

Iniciando la extracción de frames clave


c:\Users\48113164\Documents\GitHub\SignAI-IA.dev\.venv\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


[classification {
  index: 0
  score: 0.986866474
  label: "Left"
}
, classification {
  index: 1
  score: 0.956997693
  label: "Right"
}
]
[classification {
  index: 0
  score: 0.933222175
  label: "Left"
}
, classification {
  index: 1
  score: 0.926699281
  label: "Right"
}
]
num manos previas 2
Total dif 9.071300936933923
9.071300936933923
4.535650468466962
[classification {
  index: 0
  score: 0.982199
  label: "Left"
}
, classification {
  index: 1
  score: 0.957530737
  label: "Right"
}
]
num manos previas 2
Total dif 4.426340549177439
4.426340549177439
2.2131702745887196
[classification {
  index: 0
  score: 0.978651464
  label: "Left"
}
, classification {
  index: 1
  score: 0.964410841
  label: "Right"
}
]
num manos previas 2
Total dif 6.226693035749747
6.226693035749747
3.1133465178748736
[classification {
  index: 0
  score: 0.984384537
  label: "Left"
}
, classification {
  index: 1
  score: 0.95901674
  label: "Right"
}
]
num manos previas 2
Total dif 6.172700031457765
6.

In [24]:
import numpy as np

# Definir los parámetros
min_threshold = 0.075
max_threshold = 0.4
mid_size = 0.25  # Puedes ajustar este valor
k = 7.5  # Ajusta para cambiar la sensibilidad

def compute_threshold(hands_media_size):
    
    threshold = min_threshold + (max_threshold - min_threshold) / (1 + np.exp(-k * (hands_media_size - mid_size)))
    
    return threshold

# Ejemplo de uso
hands_media_size = 0.009
threshold = compute_threshold(hands_media_size)
print(f'Threshold para tamaño de manos {hands_media_size}: {threshold}')

hands_media_size = 0.05
threshold = compute_threshold(hands_media_size)
print(f'Threshold para tamaño de manos {hands_media_size}: {threshold}')

hands_media_size = 0.09
threshold = compute_threshold(hands_media_size)
print(f'Threshold para tamaño de manos {hands_media_size}: {threshold}')

hands_media_size = 0.15
threshold = compute_threshold(hands_media_size)
print(f'Threshold para tamaño de manos {hands_media_size}: {threshold}')

hands_media_size = 0.2
threshold = compute_threshold(hands_media_size)
print(f'Threshold para tamaño de manos {hands_media_size}: {threshold}')

hands_media_size = 0.3
threshold = compute_threshold(hands_media_size)
print(f'Threshold para tamaño de manos {hands_media_size}: {threshold}')

Threshold para tamaño de manos 0.009: 0.1208056770859064
Threshold para tamaño de manos 0.05: 0.1342882952370658
Threshold para tamaño de manos 0.09: 0.15022944536281926
Threshold para tamaño de manos 0.15: 0.17926692276799727
Threshold para tamaño de manos 0.2: 0.20738335501492733
Threshold para tamaño de manos 0.3: 0.26761664498507265
